In [1]:
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns 
import yfinance as yf
from datetime import datetime, timedelta
import backtrader as bt


In [2]:
plt.style.use('seaborn-v0_8')  
plt.rcParams['figure.figsize'] = (16, 8)  
plt.rcParams['font.size'] = 14  

In [3]:
# ticker_symbol = '002745.SZ'
ticker_symbol = 'GRRR'

start_date = '2022-3-27'  
end_date = '2025-3-27'

stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
print(f"The data is succesfully extracted for stock {ticker_symbol}, from{ start_date} to {end_date}，in total of {len(stock_data)} entries") 
stock_data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

The data is succesfully extracted for stock GRRR, from2022-3-27 to 2025-3-27，in total of 752 entries


Price,Close,High,Low,Open,Volume
Ticker,GRRR,GRRR,GRRR,GRRR,GRRR
Date,,,,,
2022-03-28,100.949997,101.000000,100.949997,101.0,810
2022-03-29,101.000000,101.099998,101.000000,101.0,7820
2022-03-30,101.000000,101.000000,101.000000,101.0,280
2022-03-31,100.900002,101.000000,100.900002,101.0,3240
2022-04-01,101.000000,101.000000,101.000000,101.0,220


In [4]:
df = stock_data.copy()
df.columns = stock_data.columns.droplevel(0) 
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume
0,2022-03-28,100.949997,101.000000,100.949997,101.0,810
1,2022-03-29,101.000000,101.099998,101.000000,101.0,7820
2,2022-03-30,101.000000,101.000000,101.000000,101.0,280
3,2022-03-31,100.900002,101.000000,100.900002,101.0,3240
4,2022-04-01,101.000000,101.000000,101.000000,101.0,220


In [ ]:
csv_file = f"data/{ticker_symbol}_stock_data.csv"
stock_data.to_csv(csv_file)

In [ ]:
def create_bt_data_feed(csv_file):  #
    return bt.feeds.YahooFinanceCSVData(  
        dataname=csv_file, 
        fromdate=datetime.strptime(start_date, '%Y-%m-%d'), 
        todate=datetime.strptime(end_date, '%Y-%m-%d'),
        reverse=False  
    )

In [12]:
def run_backtest(strategy_class, data_feed, cash=100000.0, commission=0.001, **strategy_params):
    
    # Initialize Backtrader engine
    cerebro = bt.Cerebro()
    
    # Add data feed
    cerebro.adddata(data_feed)
    
    # Add strategy
    cerebro.addstrategy(strategy_class, **strategy_params)
    
    # Set initial capital and commission
    cerebro.broker.setcash(cash)
    cerebro.broker.setcommission(commission=commission)
    
    # Add performance analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    
    # Run backtest
    initial_value = cerebro.broker.getvalue()
    print(f'Initial Capital: ${initial_value:.2f}')
    
    results = cerebro.run()
    strategy = results[0]
    
    
    return cerebro, strategy, results